In [82]:
from numpy import genfromtxt

motionData = genfromtxt(r'C:\Users\OndrejSpetko\Desktop\School\MED7\HRV-tracker\Breathing\motiondetection\motiondetection\out.txt', delimiter=',')
motionData.shape[0]

1440

In [74]:
FSR_IR = genfromtxt(r'C:\Users\OndrejSpetko\Desktop\School\MED7\HRV-tracker\Breathing\SerialListener\Data\1.txt', delimiter=',')
FSR_IR.shape[0]

1232

In [161]:
def scale(a):
    scalar = (np.mean(a, axis=0) - np.mean(motionData[:,1], axis=0))/np.mean(motionData[:,1], axis=0)
    #print(scalar)
    out = np.array(a, dtype=float)/scalar*30
    out -= min(out)
    return out

In [83]:
import numpy as np
import math

while motionData.shape[0] > FSR_IR.shape[0] :
    every = math.ceil(motionData.shape[0]/(motionData.shape[0] - FSR_IR.shape[0]))
    motionData = np.delete(motionData, list(range(0, motionData.shape[0], every)), axis=0)
motionData.shape

array([[ 0.4      ,  2.7495046],
       [ 0.4      ,  5.769035 ],
       [ 0.44     ,  5.769035 ],
       ...,
       [59.96     ,  4.6804237],
       [60.       ,  3.9371364],
       [60.       ,  7.2960124]])

In [92]:
from sklearn import preprocessing

# Normalize motion data
temp = np.array(motionData[:, 1])
temp.shape
motionDataNorm = preprocessing.normalize([temp])[0]
motionDataNorm

array([0.02818568, 0.05913944, 0.05913944, ..., 0.04797988, 0.04036031,
       0.07479276])

In [112]:
np.array(FSR_IR[:, 0])

array([391., 391., 391., ..., 391., 390., 390.])

In [115]:
motionData[:,1]

array([2.7495046, 5.769035 , 5.769035 , ..., 4.6804237, 3.9371364,
       7.2960124])

In [122]:
scale(np.array(FSR_IR[:, 0]))

258.8743616987067


array([1.51038518, 1.51038518, 1.51038518, ..., 1.51038518, 1.5065223 ,
       1.5065223 ])

In [162]:
# Normalize FSR data
temp = scale(np.array(FSR_IR[:, 0]))
#FSRDataNorm = preprocessing.normalize([temp])[0]
#FSRDataNorm.shape
FSRData = temp

In [163]:
# Normalize IR data
temp = scale(np.array(FSR_IR[:, 1]))
#IRDataNorm = preprocessing.normalize([temp])[0]
#IRDataNorm.shape
IRData = temp

In [164]:
#Scatter plot
import plotly as py
import plotly.graph_objs as go

# Create traces
trace0 = go.Scatter(
    x = motionData[:,0],
    y = motionData[:,1],
    mode = 'lines',
    name = 'Motion'
)
trace1 = go.Scatter(
    x = motionData[:,0],
    y = FSRData,
    mode = 'lines',
    name = 'FSR'
)
trace2 = go.Scatter(
    x = motionData[:,0],
    y = IRData,
    mode = 'lines',
    name = 'IR'
)

#data = [trace0, trace1, trace2]
data = [trace2, trace1, trace0]

# Plot and embed in ipython notebook!
py.offline.plot(data, filename='Scatter.html')

'file://C:\\Users\\OndrejSpetko\\Desktop\\School\\MED7\\HRV-tracker\\Breathing\\PostProcessing\\Python\\Scatter.html'